In [32]:
from datasets import load_dataset

EXIST_train = load_dataset("csv", data_files="EXIST2021_training.tsv", sep="\t")['train']
EXIST_test = load_dataset("csv", data_files="EXIST2021_test_labeled.tsv", sep="\t")['train']

EXIST_train = EXIST_train.remove_columns(['test_case', 'id', 'source', 'language'])
EXIST_test = EXIST_test.remove_columns(['test_case', 'id', 'source', 'language'])

EXIST_train = EXIST_train.rename_column('task1','label_sexist')
EXIST_test = EXIST_test.rename_column('task1','label_sexist')

EXIST_train = EXIST_train.rename_column('text','text_original')
EXIST_test = EXIST_test.rename_column('text','text_original')

EXIST_train = EXIST_train.class_encode_column("label_sexist")
EXIST_test = EXIST_test.class_encode_column("label_sexist")


EXIST_train = EXIST_train.filter(lambda x: len(x['text_original'])<450)
EXIST_test = EXIST_test.filter(lambda x: len(x['text_original'])<450)


In [33]:
EXIST_train[1]

{'text_original': 'Now, back to these women, the brave and the beautiful, @Clare_Crawley and @tayshia. These bad ass babes, are deserve so much credit for how this season has gone. As a woman, I’ve learned so much from them and feel more empowered to expect more in future relationships.',
 'label_sexist': 0,
 'task2': 'non-sexist'}

In [34]:
EXIST_test[1]

{'text_original': '@iilovegrapes He sounds like as ass, and very condescending.',
 'label_sexist': 0,
 'task2': 'non-sexist'}

In [35]:
import torch
from transformers import pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline(model="Helsinki-NLP/opus-mt-es-en", device=device)
# pipe = pipeline(model="Helsinki-NLP/opus-mt-zh-en")


def trans_ES_to_EN(cn: str) -> str:
    return pipe(cn,batch_size=8, truncation="only_first")[0]['translation_text']

def add_text(row):
    row['text'] = trans_ES_to_EN(row['text_original'])
    return row


In [36]:
from datasets import ClassLabel

new_features = EXIST_train.features.copy()

new_features['label_sexist'] = ClassLabel(names=["not sexist", "sexist"])
EXIST_train = EXIST_train.cast(new_features)
EXIST_test = EXIST_test.cast(new_features)


## Translate the dataset
EXIST_train = EXIST_train.map(add_text)
EXIST_test = EXIST_test.map(add_text)



## Split the dataset

split_dataset = EXIST_train.train_test_split(0.2,stratify_by_column='label_sexist')
train_dataset = split_dataset['train']
validation_dataset = split_dataset['test']
test_dataset = EXIST_test

## combine the dataset

from datasets import DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset,
})
print(dataset_dict)




## Upload the dataset

import json

file_path = "credentials.json"

with open(file_path, 'r') as json_file:
    token_data = json.load(json_file)

token = token_data.get("huggingface_token")
if token:
    print(f"Token read successfully")
else:
    print("Error: Token not found in the JSON file.")
    
import huggingface_hub

huggingface_hub.login(token)

dataset_dict.push_to_hub("yangezheng/EXIST2021")

Map:   0%|          | 0/4311 [00:00<?, ? examples/s]

/home/yzheng/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text_original', 'label_sexist', 'task2', 'text'],
        num_rows: 5564
    })
    validation: Dataset({
        features: ['text_original', 'label_sexist', 'task2', 'text'],
        num_rows: 1391
    })
    test: Dataset({
        features: ['text_original', 'label_sexist', 'task2', 'text'],
        num_rows: 4311
    })
})
Token read successfully
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/yzheng/.cache/huggingface/token
Login successful


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

In [31]:



## Upload the dataset

import json

file_path = "credentials.json"

with open(file_path, 'r') as json_file:
    token_data = json.load(json_file)

token = token_data.get("huggingface_token")
if token:
    print(f"Token read successfully")
else:
    print("Error: Token not found in the JSON file.")
    
import huggingface_hub

huggingface_hub.login(token)

dataset_dict.push_to_hub("yangezheng/EXIST2021")

Token read successfully
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/yzheng/.cache/huggingface/token
Login successful


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]